In [ ]:
#from collections import defaultdict
#import array
#import csv
import numpy as np
#from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
#import itertools
import pandas as pd
from random import sample, randint

In [ ]:
dbb = pd.read_csv("time_fixed_DB.csv", encoding="UTF-8",sep=";")
dff= pd.read_csv("time_fixed_DF.csv", encoding="UTF-8",sep=";")
weight = 1/np.log10(dff.loc[:,"movieId"].value_counts())

In [ ]:
db = dbb.copy()
nb_movies = 30
iteration = 100
moviessss = list(db.loc[:,"movieId"].unique())

resultt = {}
for nb_movies in range(1,31):
    sumecc = 0
    new_NF_userId = [2649430 for i in range(nb_movies)]
    new_ML_userId = [138494 for i in range(nb_movies)]
    for i in range(iteration):
        db = dbb.copy()
        selected_movies = sample(moviessss,nb_movies)
        new_rates = [randint(1,5) for i in range(nb_movies)]
        new_timestamp = [randint(0,5000) for i in range(nb_movies)]
        db = db.append(pd.DataFrame({'userId': new_ML_userId,'movieId':selected_movies,'rating':new_rates,"timestamp":new_timestamp}),ignore_index=True)
        df = pd.DataFrame({'userId': new_NF_userId,'movieId':selected_movies,'rating':new_rates,"timestamp":new_timestamp})
        db.rename(columns={"userId":"uM"},inplace=1)
        df.rename(columns={"userId":"uF"},inplace=1)
        db.set_index(["movieId"],inplace=True)
        df.set_index(["movieId"],inplace=True)
        com_movies = df.index.unique().intersection(db.index.unique()) #Ordering, comment if working on all datasets
        db = db.loc[com_movies,:]
        df = df.loc[com_movies,:]
        res = df.subtract(db,fill_value=0).astype(np.int64)
        res = np.abs(res)
        result = weight.loc[com_movies]*(np.exp(-res.loc[:,'rating']/1.5) + np.exp(-res.loc[:,'timestamp']/30))
        res.loc[:,"score"] = result
        res.drop(["rating", "timestamp"],axis=1,inplace=True)
        res.set_index(['uF','uM'],inplace=True)
        R = res.groupby(level=[0,1]).sum()
        values = R.index.levels[0]
        for elem in values:
            tempUser = R.loc[elem]
            tempScore = np.array(tempUser["score"])
            stdev = np.std(tempScore)
            best1 = np.max(tempScore)
            idx1 = np.argmax(tempScore)
            tempScore = np.delete(tempScore, idx1)
            best2 = np.max(tempScore)
            ecc = (best1-best2)/stdev
            sumecc += ecc
    resultt[nb_movies] = sumecc/iteration

In [ ]:
db = dbb.copy()
nb_movies = 30
iteration = 100
moviessss = list(db.loc[:,"movieId"].unique())
resultt = []
new_NF_userId = [2649430 for i in range(nb_movies)]
new_ML_userId = [138494 for i in range(nb_movies)]
for noise in range(1,11):
    sumecc = 0
    for i in range(iteration):
        db = dbb.copy()
        selected_movies = sample(moviessss,nb_movies)
        new_rates = [randint(1,5) for i in range(nb_movies)]
        new_ratesNF = new_rates +  [randint(-noise,noise) for i in range(nb_movies)]/2
        new_ratesNF[new_ratesNF > 5] = 5
        new_ratesNF[new_ratesNF < 1] = 1
        new_timestamp = [randint(0,5000) for i in range(nb_movies)]
        db = db.append(pd.DataFrame({'userId': new_ML_userId,'movieId':selected_movies,'rating':new_ratesNF,"timestamp":new_timestamp}),ignore_index=True)
        df = pd.DataFrame({'userId': new_NF_userId,'movieId':selected_movies,'rating':new_rates,"timestamp":new_timestamp})
        db.rename(columns={"userId":"uM"},inplace=1)
        df.rename(columns={"userId":"uF"},inplace=1)
        db.set_index(["movieId"],inplace=True)
        df.set_index(["movieId"],inplace=True)
        com_movies = df.index.unique().intersection(db.index.unique()) #Ordering, comment if working on all datasets
        db = db.loc[com_movies,:]
        df = df.loc[com_movies,:]
        res = df.subtract(db,fill_value=0).astype(np.int64)
        res = np.abs(res)
        result = weight.loc[com_movies]*(np.exp(-res.loc[:,'rating']/1.5) + np.exp(-res.loc[:,'timestamp']/30))
        res.loc[:,"score"] = result
        res.drop(["rating", "timestamp"],axis=1,inplace=True)
        res.set_index(['uF','uM'],inplace=True)
        R = res.groupby(level=[0,1]).sum()
        values = R.index.levels[0]
        for elem in values:
            tempUser = R.loc[elem]
            tempScore = np.array(tempUser["score"])
            stdev = np.std(tempScore)
            best1 = np.max(tempScore)
            idx1 = np.argmax(tempScore)
            tempScore = np.delete(tempScore, idx1)
            best2 = np.max(tempScore)
            ecc = (best1-best2)/stdev
            sumecc += ecc
    resultt.append(sumecc/iteration)

In [ ]:
import matplotlib.pyplot as plt
a = np.zeros((31,1))
for key in resultt:
    a[key] = resultt[key]
display(a)
plt.plot(a)
plt.show()

In [ ]:
plt.plot(resultt)
plt.show()
display(dbb["movieId"].unique().shape)
display(weight.shape)

In [ ]:
import csv
with open('nb_movies.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerows(a)

In [ ]:
display(df.head())
display(db.loc[db['uM']==138494].head())